TODO

1. Conferir perguntas Decision Stump
2. Adicionar Pesos dos pontos

In [1]:
import os, time

import numpy as np
import matplotlib as plt
import pandas as pd

pd.options.display.max_rows = 10

In [16]:
data = pd.read_csv('data/tic-tac-toe.data', header=None)
data

,0,1,2,3,4,5,6,7,8,9
0,x,x,x,x,o,o,x,o,o,positive
1,x,x,x,x,o,o,o,x,o,positive
2,x,x,x,x,o,o,o,o,x,positive
3,x,x,x,x,o,o,o,b,b,positive
4,x,x,x,x,o,o,b,o,b,positive
...,...,...,...,...,...,...,...,...,...,...
953,o,x,x,x,o,o,o,x,x,negative
954,o,x,o,x,x,o,x,o,x,negative
955,o,x,o,x,o,x,x,o,x,negative
956,o,x,o,o,x,x,x,o,x,negative


In [17]:
# Armazenar o label correto dos numeros
data = data.rename(columns = {9:'label'})
data['label'] = data['label'].replace({'positive': 1, 'negative': -1})

# Separar 20% para teste
train = data.sample(frac = 0.8)
test = data.drop(train.index)

y_train = train.label
y_test  = test.label

# Retirar a primeira coluna referente aos labels
X_train = train.drop("label", axis=1)
X_test  = test.drop("label", axis=1)

In [32]:
class DecisionStump:
    def __init__(self):
        # Possiveis categorias que o dado vai ter para poder realizar as perguntas (Tem x na posição 0?)
        # No caso do tick-tack-toe vai ser x, o e b
        self.unique_categories = []
        # Dimensionalidade do dado, para saber quantas possíveis posições pode ter
        self.data_dimension = 0
        
        self.model = {
            'question_column' : None,
            'question_cat'    : None,
            'value'           : None,
            'train_error'     : None
        }
        
    def fit(self, X, y, weights=None):
        """Seleciona o modelo com base em X, y e os pesos de cada entrada"""
        
        # Guarda todos os valores possiveis para todas as colunas e a dimensao dos dados
        # No caso do Tick-Tack-Toe, sera x,o,b para todas
        for column in X:
            self.unique_categories.append(X[column].unique())
        self.data_dimension = len(list(X))
        
        best_model_error = 2
        for i in range(0, self.data_dimension):
            for cat in self.unique_categories[0]:
                for value in (-1, 1):
                    error = self._compute_training_error(X, y, i, cat, value)
                    if (error < best_model_error):
                        best_model_error  = error
                        self.model['question_column'] = i
                        self.model['question_cat']    = cat
                        self.model['value']           = value
                        self.model['train_error']     = error
                        print('Best model {}-{}-{} with error {}'.format(
                            self.model['question_column'], self.model['question_cat'],
                            self.model['value'], error))
        
        print('Best model {} {} {} {}'.format(self.model['question_column'], self.model['question_cat'],
                                        self.model['value'], self.model['train_error']))
        
        return
    
    def predict(self, X):
        """Prevê os valores das entradas em X"""
        if (self.model['question_column'] == None):
            print('Model was not fitted')
            return
        
        predictions = []
        for row in X:
            if( row[self.model['question_column']] == self.model['question_cat'] ):
                predictions.append(self.model['value'])
            else:
                predictions.append(-self.model['value'])
        return predictions
    
    def _compute_training_error(self, X, y, question_column, question_cat, value, weights=None):
        nb_samples  = X.count()[0]
        right_count = 0
        for i, row in X.iterrows():
            if( (row[question_column] == question_cat and y[i] == value) or 
                  (row[question_column] != question_cat and y[i] != value) ):
                right_count += 1
        
        return 1 - (right_count / nb_samples)
        
    
dt = DecisionStump()
dt.fit(X_train, y_train)

predictions = dt.predict(X_test.values)
right_count = 0
for i, row in enumerate(predictions):
    if (predictions[i] == y_test.values[i]):
        right_count += 1
print('Right count {}'.format(right_count / float(len(y_test.values))) )

Best model 0-x--1 with error 0.5078328981723237
Best model 0-x-1 with error 0.4921671018276762
Best model 0-b--1 with error 0.4268929503916449
Best model 0-o--1 with error 0.40731070496083555
Best model 2-o--1 with error 0.3968668407310705
Best model 4-x-1 with error 0.3590078328981723
Best model 4-o--1 with error 0.2924281984334204
Best model 4 o -1 0.2924281984334204
Right count 0.6666666666666666
